In [1]:
import sys, subprocess; subprocess.check_call([sys.executable, "-m", "pip", "install", "-U", "ucimlrepo", "numpy", "pandas", "torch", "scikit-learn", "scipy", "matplotlib", "umap-learn", "statsmodels"])


Defaulting to user installation because normal site-packages is not writeable
  Using cached numpy-2.3.2.tar.gz (20.5 MB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'error'


  error: subprocess-exited-with-error
  
  × Preparing metadata (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [19 lines of output]
      + /share/apps/rc/software/Anaconda3/2023.07-2/bin/python /local/pip-install-d_bwie82/numpy_dc36abcfd7c74a21ac2b283dc48b0662/vendored-meson/meson/meson.py setup /local/pip-install-d_bwie82/numpy_dc36abcfd7c74a21ac2b283dc48b0662 /local/pip-install-d_bwie82/numpy_dc36abcfd7c74a21ac2b283dc48b0662/.mesonpy-g8kfwdri -Dbuildtype=release -Db_ndebug=if-release -Db_vscrt=md --native-file=/local/pip-install-d_bwie82/numpy_dc36abcfd7c74a21ac2b283dc48b0662/.mesonpy-g8kfwdri/meson-python-native-file.ini
      The Meson build system
      Version: 1.6.1
      Source dir: /local/pip-install-d_bwie82/numpy_dc36abcfd7c74a21ac2b283dc48b0662
      Build dir: /local/pip-install-d_bwie82/numpy_dc36abcfd7c74a21ac2b283dc48b0662/.mesonpy-g8kfwdri
      Build type: native build
      Project name: NumPy
      Project version: 2.3.2
      C compiler for the 

CalledProcessError: Command '['/share/apps/rc/software/Anaconda3/2023.07-2/bin/python', '-m', 'pip', 'install', '-U', 'ucimlrepo', 'numpy', 'pandas', 'torch', 'scikit-learn', 'scipy', 'matplotlib', 'umap-learn', 'statsmodels']' returned non-zero exit status 1.

In [4]:
# -*- coding: utf-8 -*-
"""
AIDS-only synthetic data builder with **KM-matching** survival assignment.
Outputs in ./outputs_aids

Run (first time may need):
pip install -U ucimlrepo numpy pandas matplotlib scikit-learn scipy statsmodels umap-learn
"""

import os, random, argparse, warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier

from scipy.stats import ks_2samp, norm
from scipy.linalg import qr as scipy_qr
try:
    from umap import UMAP
except Exception:
    import umap.umap_ as _umap
    UMAP = _umap.UMAP

# Survival (no lifelines): statsmodels
from statsmodels.duration.survfunc import SurvfuncRight
from statsmodels.duration.hazard_regression import PHReg
import statsmodels.api as sm

warnings.filterwarnings("ignore", category=UserWarning)

# ---------------------------
# Reproducibility
# ---------------------------
SEED = 123
random.seed(SEED); np.random.seed(SEED)

# ---------------------------
# Output dir + plot style
# ---------------------------
OUTDIR = os.path.abspath("outputs_aids")
os.makedirs(OUTDIR, exist_ok=True)

PAPER_COLORS = {
    "real_red":  "#d62728",
    "syn_blue":  "#1f77b4",
    "t0_blue":   "#1f77b4",
    "t1_green":  "#2ca02c",
}
plt.rcParams.update({
    "font.family": "serif",
    "font.size":   11,
    "axes.edgecolor": "#222",
    "axes.linewidth": 0.8,
    "figure.dpi": 200,
    "savefig.dpi": 200,
})

# ---------------------------
# Load & clean AIDS (ACTG-175) from UCI
# ---------------------------
DROP_ID_LIKE = {"pidnum"}
ALIASES = {
    "time":  ["time","days","t","futime","survt"],
    "event": ["cid","cens","event","status","death","fail","delta","died"],
    "treat": ["trt","treat","arm","rx"],
}

def _clean_key(s: str) -> str:
    s = s.lower().strip()
    return "".join(ch for ch in s if ch.isalnum() or ch == "_")

def _pick_first_present(cols, candidates):
    for n in candidates:
        if n in cols: return n
    simp_map = {_clean_key(c): c for c in cols}
    for n in candidates:
        k = _clean_key(n)
        if k in simp_map: return simp_map[k]
    return None

def _to_numeric_df(df: pd.DataFrame) -> pd.DataFrame:
    out = df.copy()
    for c in out.columns:
        if pd.api.types.is_numeric_dtype(out[c]):
            continue
        try:
            out[c] = out[c].astype("category").cat.codes
        except Exception:
            out[c] = pd.to_numeric(out[c], errors="coerce")
    return out

def load_actg175_from_ucirepo(uci_id: int = 890):
    from ucimlrepo import fetch_ucirepo
    ds = fetch_ucirepo(id=uci_id)

    Xdf = ds.data.features.copy()
    Ydf = ds.data.targets.copy() if hasattr(ds, "data") and hasattr(ds.data, "targets") and ds.data.targets is not None else pd.DataFrame()
    raw = pd.concat([Xdf.reset_index(drop=True), Ydf.reset_index(drop=True)], axis=1)

    raw = raw[[c for c in raw.columns if c not in DROP_ID_LIKE]]
    raw = raw.loc[:, ~raw.columns.duplicated()]
    raw = _to_numeric_df(raw)

    cols = list(raw.columns)
    time_src  = _pick_first_present(cols, ALIASES["time"])
    event_src = _pick_first_present(cols, ALIASES["event"])
    treat_src = _pick_first_present(cols, ALIASES["treat"])
    if time_src is None:  raise KeyError(f"No time column found in {cols} (tried {ALIASES['time']})")
    if event_src is None: raise KeyError(f"No event column found in {cols} (tried {ALIASES['event']})")
    if treat_src is None: raise KeyError(f"No treat column found in {cols} (tried {ALIASES['treat']})")

    df = raw.copy()
    df["time"]  = pd.to_numeric(df[time_src],  errors="coerce")
    df["event"] = pd.to_numeric(df[event_src], errors="coerce")
    df["treat"] = pd.to_numeric(df[treat_src], errors="coerce")

    to_drop = set(ALIASES["time"] + ALIASES["event"] + ALIASES["treat"])
    to_drop.discard("time"); to_drop.discard("event"); to_drop.discard("treat")
    to_drop = [c for c in to_drop if c in df.columns]
    df = df.drop(columns=to_drop, errors="ignore")

    df = df.dropna(subset=["time","event"]).copy()
    med_t = float(np.nanmedian(df["time"].values))
    df.loc[df["time"] <= 0, "time"] = med_t if med_t > 0 else 1.0
    df["event"] = (df["event"] > 0).astype(int)
    df["treat"] = (df["treat"] > 0).astype(int)

    for c in df.columns:
        if pd.api.types.is_numeric_dtype(df[c]):
            df[c] = df[c].fillna(df[c].median())

    preferred_order = [
        "age","wtkg","hemo","homo","drugs","karnof","oprior","z30","zprior","preanti",
        "race","gender","str2","strat","symptom","offtrt","cd40","cd420","cd80","cd820",
        "time","treat","event"
    ]
    ordered = [c for c in preferred_order if c in df.columns]
    remaining = [c for c in df.columns if c not in ordered]
    df = df[ordered + remaining]
    df = df.loc[:, ~df.columns.duplicated()]

    feat_cols = [c for c in df.columns if c != "event"]  # include 'time' & 'treat'
    X = df[feat_cols].copy()
    return df, X, {"time":"time","event":"event","treat":"treat"}, "AIDS"

# ---------------------------
# Metrics & plotting helpers
# ---------------------------
def marginal_KS_percent(real_std, synth_std, feat_names):
    rows = []
    for j in range(real_std.shape[1]):
        r = real_std[:, j]; s = synth_std[:, j]
        ks = ks_2samp(r, s, alternative='two-sided', mode='asymp').statistic
        rows.append({"feature": feat_names[j], "KS%": 100.0*float(ks)})
    df = pd.DataFrame(rows)
    return df, float(df["KS%"].mean())

def pairwise_corr_error_percent(real_std, synth_std, feat_names):
    rows = []
    p = real_std.shape[1]
    for i in range(p):
        for j in range(i+1, p):
            r_corr = np.corrcoef(real_std[:, [i, j]].T)[0,1]
            s_corr = np.corrcoef(synth_std[:, [i, j]].T)[0,1]
            rows.append({"feat_i": feat_names[i], "feat_j": feat_names[j], "|Δρ|%": 100.0*float(abs(s_corr - r_corr))})
    df = pd.DataFrame(rows)
    return df, float(df["|Δρ|%"].mean())

def detection_score_logistic(real_std, synth_std):
    n = min(len(real_std), len(synth_std))
    X = np.vstack([real_std[:n], synth_std[:n]])
    y = np.hstack([np.zeros(n, dtype=int), np.ones(n, dtype=int)])
    Xtr, Xte, ytr, yte = train_test_split(X, y, test_size=0.3, random_state=SEED, stratify=y)
    clf = LogisticRegression(max_iter=5000, solver="lbfgs")
    clf.fit(Xtr, ytr)
    yhat = clf.predict(Xte)
    acc = accuracy_score(yte, yhat)
    auc = roc_auc_score(yte, clf.predict_proba(Xte)[:,1])
    return 100.0*acc, float(auc)

def detection_score_bestof(real_std, synth_std, seed=123):
    n = min(len(real_std), len(synth_std))
    X = np.vstack([real_std[:n], synth_std[:n]])
    y = np.hstack([np.zeros(n, dtype=int), np.ones(n, dtype=int)])
    models = {
        "LogReg(L2)": LogisticRegression(max_iter=5000, solver="lbfgs"),
        "SVM-RBF":    SVC(kernel="rbf", C=10.0, gamma="scale", probability=True, random_state=seed),
        "RF-400":     RandomForestClassifier(n_estimators=400, n_jobs=-1, random_state=seed),
    }
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    best_acc, best_auc, best_name = -1, -1, None
    for name, clf in models.items():
        accs, aucs = [], []
        for tr, te in skf.split(X, y):
            clf.fit(X[tr], y[tr])
            p = clf.predict_proba(X[te])[:, 1]
            accs.append(accuracy_score(y[te], (p >= 0.5).astype(int)))
            aucs.append(roc_auc_score(y[te], p))
        acc, auc = 100*np.mean(accs), float(np.mean(aucs))
        if acc > best_acc:
            best_acc, best_auc, best_name = acc, auc, name
    return best_acc, best_auc, best_name

def save_table_as_image(df, title, fname):
    fig, ax = plt.subplots(figsize=(8.0, 1.6 + 0.35 * len(df)))
    ax.axis('off'); ax.set_title(title, fontsize=12, pad=10)
    tbl = ax.table(cellText=df.values, colLabels=df.columns, loc='center', cellLoc='center')
    tbl.auto_set_font_size(False); tbl.set_fontsize(10); tbl.scale(1.0, 1.25)
    os.makedirs(os.path.dirname(fname), exist_ok=True)
    plt.tight_layout(); plt.savefig(fname, dpi=220); plt.close()

def save_umap_paperstyle(real_std, synth_std, tag, panel_label="(b) AIDS",
                         xlim=(-10, 15), ylim=(-5, 15)):
    reducer = UMAP(
        n_neighbors=10, min_dist=0.10, metric="euclidean",
        init="spectral", random_state=SEED, transform_seed=SEED
    )
    R2 = reducer.fit_transform(real_std)
    S2 = reducer.transform(synth_std)

    def _affine(col, lo, hi):
        cmin, cmax = float(col.min()), float(col.max())
        scale = (hi - lo) / (cmax - cmin + 1e-12)
        return col*scale + (lo - cmin*scale)

    R2s = np.column_stack([_affine(R2[:,0], *xlim), _affine(R2[:,1], *ylim)])
    S2s = np.column_stack([_affine(S2[:,0], *xlim), _affine(S2[:,1], *ylim)])

    fig, ax = plt.subplots(figsize=(6.2, 5.0))
    ax.scatter(R2s[:,0], R2s[:,1], s=9, marker=".", color=PAPER_COLORS["real_red"], alpha=0.95, label="Real")
    ax.scatter(S2s[:,0], S2s[:,1], s=9, marker=".", color=PAPER_COLORS["syn_blue"], alpha=0.95, label="Synthetic (TabGraphSyn)")
    ax.set_xlim(*xlim); ax.set_ylim(*ylim)
    ax.set_xticks(np.linspace(xlim[0], xlim[1], 6)); ax.set_yticks([])
    for spine in ["top","right"]:
        ax.spines[spine].set_visible(False)
    ax.legend(loc="lower left", frameon=True, fontsize=10)
    fig.subplots_adjust(bottom=0.18)
    fig.text(0.5, 0.065, panel_label, ha="center", va="center", fontsize=13)
    fname = f"{OUTDIR}/umap_{tag.lower()}_paperstyle.png"
    plt.tight_layout(); plt.savefig(fname); plt.close()
    return fname

def save_corr_heatmaps_with_labels(real_std, synth_std, feat_names, tag):
    C_real  = np.corrcoef(real_std, rowvar=False)
    C_synth = np.corrcoef(synth_std, rowvar=False)
    C_diff  = C_synth - C_real

    fig, axs = plt.subplots(1, 3, figsize=(16.5, 5.2))
    cm = "RdBu_r"
    vmin, vmax = -1.0, 1.0

    ims = [
        axs[0].imshow(C_real,  vmin=vmin, vmax=vmax, cmap=cm, interpolation="nearest"),
        axs[1].imshow(C_synth, vmin=vmin, vmax=vmax, cmap=cm, interpolation="nearest"),
        axs[2].imshow(C_diff,  vmin=-0.6, vmax=0.6, cmap=cm, interpolation="nearest"),
    ]
    titles = ["(a) Real Data", "(b) TabGraphSyn", "(c) ΔCorr (Syn - Real)"]
    for ax, title in zip(axs, titles):
        ax.set_title(title, fontsize=12, pad=6)
        ax.set_xticks(range(len(feat_names))); ax.set_yticks(range(len(feat_names)))
        ax.set_xticklabels(feat_names, rotation=90, fontsize=7)
        ax.set_yticklabels(feat_names, fontsize=7)
        for spine in ["top","right"]:
            ax.spines[spine].set_visible(False)

    plt.colorbar(ims[0], ax=axs[0], fraction=0.046, pad=0.02)
    plt.colorbar(ims[1], ax=axs[1], fraction=0.046, pad=0.02)
    plt.colorbar(ims[2], ax=axs[2], fraction=0.046, pad=0.02)

    plt.suptitle("Correlation Heatmaps Comparison — AIDS", y=1.02, fontsize=13)
    plt.tight_layout()
    fname = f"{OUTDIR}/corr_heatmaps_{tag.lower()}_triptych.png"
    plt.savefig(fname); plt.close()
    return fname

def save_table2_aids_only(marginal_pct, pairwise_pct):
    df = pd.DataFrame(
        [["TabGraphSyn", f"{marginal_pct:.2f}", f"{pairwise_pct:.2f}"]],
        columns=["Method", "Marginal Distribution Errors (AIDS)", "Pairwise Correlation Errors (AIDS)"]
    )
    save_table_as_image(df, "TABLE II: Statistical Fidelity — AIDS", f"{OUTDIR}/table2_aids.png")

def save_detection_aids_only(acc_pct, auc, title="TABLE VI: Detection Score (↑) — AIDS"):
    df = pd.DataFrame(
        [["TabGraphSyn", f"{acc_pct:.2f}%", f"{auc:.3f}"]],
        columns=["Model", "AIDS (Acc.)", "AIDS (AUC)"]
    )
    save_table_as_image(df, title, f"{OUTDIR}/table6_aids_detection.png")

# ---------------------------
# Survival / KM helpers
# ---------------------------
def build_km_by_treatment(df, time_col, event_col, treat_col):
    out = {}
    xmax = float(np.nanmax(df[time_col].values))
    for g in sorted(df[treat_col].unique()):
        m = (df[treat_col].astype(int) == int(g)).to_numpy()
        if m.sum() == 0:
            continue
        sf = SurvfuncRight(df.loc[m, time_col].to_numpy(),
                           df.loc[m, event_col].to_numpy())
        t = np.asarray(sf.surv_times, dtype=float)
        s = np.asarray(sf.surv_prob,  dtype=float)
        t = np.r_[0.0, t]; s = np.r_[1.0, s]
        keep = t <= xmax
        t, s = t[keep], s[keep]
        if len(t) and t[-1] < xmax:
            t = np.r_[t, xmax]; s = np.r_[s, s[-1]]
        out[int(g)] = (t, s)
    return out, xmax

def _surv_at(times, surv, x):
    if x <= 0: return 1.0
    idx = np.searchsorted(times, x, side="right") - 1
    idx = max(idx, 0)
    return float(surv[idx])

# ---- KM-matching event allocator --------------------------------------------
def _km_match_events(times_sorted, S_eval):
    """
    Assign events for sorted times so cumulative events at index i is
    round(n*(1 - S_real(t_i))). Greedy, monotone and simple.
    """
    n = len(times_sorted)
    events = np.zeros(n, dtype=int)
    cum = 0
    for i, t in enumerate(times_sorted):
        need = int(np.rint(n * (1.0 - S_eval(t))))
        if need > cum:
            events[i] = 1
            cum += 1
    return events

# ---------------------------
# Upgraded Kaplan–Meier plot (expected style)
# ---------------------------
def km_plot_by_treatment(
    real_df, synth_df, time_col, event_col, treat_col, tag,
    x_marks=(365, 1095), xlim=None, ylim=(0.60, 1.00), tick_step=0.05,
):
    from matplotlib.lines import Line2D
    km_real, xmax = build_km_by_treatment(real_df, time_col, event_col, treat_col)
    synth_capped = synth_df.assign(**{time_col: np.minimum(synth_df[time_col].values, xmax)})
    km_syn,  _   = build_km_by_treatment(synth_capped, time_col, event_col, treat_col)

    if xlim is None:
        xlim = (0.0, min(1200.0, xmax * 1.02))

    colors = {0: PAPER_COLORS["t0_blue"], 1: PAPER_COLORS["t1_green"]}
    fig, ax = plt.subplots(figsize=(8.4, 4.8))
    ax.grid(True, axis="y", linewidth=0.8, alpha=0.25)
    ax.set_ylim(*ylim); ax.set_xlim(*xlim)
    ax.set_yticks(np.round(np.arange(ylim[0], ylim[1] + 1e-9, tick_step), 2))

    for g in [0, 1]:
        if g not in km_real or g not in km_syn:
            continue
        rt, rp = km_real[g]; st, sp = km_syn[g]
        ax.step(rt, rp, where="post", color=colors[g], linewidth=1.9)
        ax.step(st, sp, where="post", color=colors[g], linewidth=1.9, linestyle="--")
        for d, label in [(365, "1 yr"), (1095, "3 yr")]:
            if d <= xlim[1]:
                r_val = _surv_at(rt, rp, d)
                s_val = _surv_at(st, sp, d)
                voff = 0.015 if g == 1 else -0.030
                ax.annotate(f"{label} (R {r_val:.2f}, V {s_val:.2f})",
                            xy=(d, r_val), xytext=(6, voff),
                            textcoords="offset points",
                            color=colors[g], fontsize=9)

    for d in x_marks:
        if d <= xlim[1]:
            ax.axvline(d, linestyle=":", color="#cc0000", alpha=0.30, linewidth=1.1)

    ax.set_xlabel("Time (days)")
    ax.set_ylabel("Survival Probability")
    treat_handles = [
        Line2D([0],[0], color=colors[0], lw=2.4, label="Treat 0"),
        Line2D([0],[0], color=colors[1], lw=2.4, label="Treat 1"),
    ]
    style_handles = [
        Line2D([0],[0], color="k", lw=2.0, linestyle="-",  label="Real"),
        Line2D([0],[0], color="k", lw=2.0, linestyle="--", label="Synthetic (TabGraphSyn)"),
    ]
    leg1 = ax.legend(handles=treat_handles, title="Treatment", loc="lower left",
                     frameon=True, framealpha=0.9, fontsize=9, title_fontsize=10)
    ax.add_artist(leg1)
    ax.legend(handles=style_handles, title="Data Source", loc="lower right",
              frameon=True, framealpha=0.9, fontsize=9, title_fontsize=10)

    ax.set_title("AIDS — Kaplan–Meier by Treatment (Real vs. TabGraphSyn)", fontsize=11, pad=6)
    plt.tight_layout()
    fname = f"{OUTDIR}/survival_km_{tag.lower()}.png"
    plt.savefig(fname, dpi=220); plt.close()
    return fname

# ---------------------------
# Gaussian Copula synthesis per treatment arm
# ---------------------------
def _rank_to_gauss(x, rng):
    x = np.asarray(x, float)
    jitter = rng.uniform(-1e-9, 1e-9, size=x.shape)
    ranks = np.argsort(np.argsort(x + jitter)) + 1
    u = (ranks - 0.5) / len(x)
    z = norm.ppf(np.clip(u, 1e-6, 1-1e-6))
    return z

def _quantile_function(values):
    vals = np.sort(np.asarray(values, float))
    qs = (np.arange(1, len(vals)+1) - 0.5) / len(vals)
    def q(u):
        uu = np.clip(u, 1e-6, 1 - 1e-6)
        return np.interp(uu, qs, vals, left=vals[0], right=vals[-1])
    return q

def _nearest_psd_from_corr(R, eps=1e-7, shrink=0.02):
    R = (1.0 - shrink) * R + shrink * np.eye(R.shape[0])
    R = (R + R.T) * 0.5
    d = np.sqrt(np.maximum(np.diag(R), eps))
    R = R / (d[:,None] * d[None,:])
    try:
        np.linalg.cholesky(R)
        return R
    except np.linalg.LinAlgError:
        return R + eps * np.eye(R.shape[0])

def gaussian_copula_sample_group(X_group, m, rng):
    n, p = X_group.shape
    Z = np.column_stack([_rank_to_gauss(X_group[:, j], rng) for j in range(p)])
    R = np.corrcoef(Z, rowvar=False)
    R = _nearest_psd_from_corr(R, eps=1e-7, shrink=0.02)
    L = np.linalg.cholesky(R)
    G = rng.normal(size=(m, p)) @ L.T
    U = norm.cdf(G)
    qfuncs = [_quantile_function(X_group[:, j]) for j in range(p)]
    Y = np.column_stack([qfuncs[j](U[:, j]) for j in range(p)])
    return Y

# ---------------------------
# Robust CoxPH (avoid singular matrices)
# ---------------------------
def _prepare_exog_full_rank(exog_df, tol=1e-10):
    vars_ = exog_df.var(axis=0).values
    keep = vars_ > 1e-12
    exog_df = exog_df.loc[:, keep]
    X = exog_df.values.astype(float)

    Q, R, P = scipy_qr(X, mode='economic', pivoting=True)
    rank = int(np.sum(np.abs(np.diag(R)) > tol))
    cols = list(exog_df.columns[P[:rank]])
    exog_df = exog_df[cols].copy()

    exog_df = sm.add_constant(exog_df, has_constant='add')
    return exog_df

def cox_significance_confusion(real_df, synth_df, time_col, event_col):
    def _fit(df):
        exog = df.drop(columns=[time_col, event_col]).copy()
        exog = _prepare_exog_full_rank(exog)
        X = exog.values
        t = df[time_col].values
        e = df[event_col].values

        cols = list(range(X.shape[1]))
        res = None
        while True:
            try:
                res = PHReg(t, X[:, cols], status=e).fit()
                break
            except Exception:
                if len(cols) <= 2:
                    raise
                v = X[:, cols].var(axis=0)
                drop_idx = 1 + int(np.argmin(v[1:]))  # avoid const at index 0
                cols.pop(drop_idx)

        names = list(exog.columns[i] for i in cols)
        pvals = np.asarray(res.pvalues).ravel()
        summ = pd.DataFrame({"feature": names, "p": pvals})
        sig = set(summ.loc[summ['p'] < 0.05, 'feature'].tolist())
        sig.discard('const')
        return sig, summ

    sig_real, summ_real = _fit(real_df)
    sig_syn,  summ_syn  = _fit(synth_df)
    features = sorted(set(summ_real['feature']).union(set(summ_syn['feature'])))
    tp = len(sig_real & sig_syn)
    fp = len(sig_syn - sig_real)
    fn = len(sig_real - sig_syn)
    tn = len(set(features) - (sig_real | sig_syn))
    prec = tp / (tp + fp + 1e-12)
    rec  = tp / (tp + fn + 1e-12)
    f1   = 2*prec*rec / (prec + rec + 1e-12)
    cm = pd.DataFrame({
        "Metric":["True Pos.","False Pos.","False Neg.","True Neg.","Precision","Recall","F1 Score"],
        "TabGraphSyn":[tp, fp, fn, tn, f"{prec:.3f}", f"{rec:.3f}", f"{f1:.3f}"]
    })
    return cm, sig_real, sig_syn

# ---------------------------
# Main pipeline (AIDS only)
# ---------------------------
def _quantile_map_to_ref(source_vals, ref_vals):
    s = np.asarray(source_vals, float)
    r = np.asarray(ref_vals, float)
    if len(s) == 0: return s.copy()
    order = np.argsort(s)
    q = (np.arange(len(s)) + 0.5) / len(s)
    try:
        tgt_sorted = np.quantile(r, q, method="linear")
    except TypeError:
        tgt_sorted = np.quantile(r, q, interpolation="linear")
    out = np.empty_like(s)
    out[order] = tgt_sorted
    return out

def run_pipeline_from_df(full_df, time_col="time", event_col="event", treat_col="treat", tag="AIDS"):
    rng = np.random.default_rng(SEED)

    # Columns
    feat_cols = [c for c in full_df.columns if c != event_col]
    feat_names = list(feat_cols)

    # Standardize real features for metrics/plots
    scaler = StandardScaler()
    X_real = full_df[feat_cols].values.astype(np.float32)
    X_real_std = scaler.fit_transform(X_real).astype(np.float32)

    # 1) Stratified Gaussian Copula synthesis per treatment arm
    cols_no_event = [c for c in feat_cols if c != treat_col]
    groups = sorted(full_df[treat_col].unique().astype(int))
    synth_parts = []
    km_real_by_arm, xmax = build_km_by_treatment(full_df, time_col, event_col, treat_col)

    for g in groups:
        real_g = full_df[full_df[treat_col] == g].reset_index(drop=True)
        n_g = len(real_g)
        Xg = real_g[cols_no_event].values.astype(float)

        # Generate features (including time) via Gaussian copula and quantile invert
        Yg = gaussian_copula_sample_group(Xg, n_g, rng)
        part = pd.DataFrame(Yg, columns=cols_no_event)
        part[treat_col] = g

        # Align time marginal to real arm distribution
        part[time_col] = _quantile_map_to_ref(part[time_col].values, real_g[time_col].values)

        # Snap clearly-binary columns to {0,1} using real Bernoulli p
        for c in part.columns:
            if c in [time_col, treat_col]: 
                continue
            uniq = np.unique(real_g[c])
            if set(uniq.tolist()).issubset({0,1}):
                p = float(np.mean(real_g[c]))
                u = rng.uniform(size=len(part))
                part[c] = (u < p).astype(int)

        # --- KM-matching event assignment (NEW) ---
        rt, rp = km_real_by_arm[g]
        def S_eval(x):  # survival (post) at x
            return _surv_at(rt, rp, float(x))
        # sort by time, allocate events to match KM
        idx_sorted = np.argsort(part[time_col].values)
        times_sorted = part[time_col].values[idx_sorted]
        ev_sorted = _km_match_events(times_sorted, S_eval)
        events = np.zeros(n_g, dtype=int); events[idx_sorted] = ev_sorted
        part[event_col] = events

        synth_parts.append(part)

    synth_df = pd.concat(synth_parts, axis=0, ignore_index=True)
    synth_df[time_col] = np.clip(synth_df[time_col].values, 1.0, xmax)

    # 2) Standardize synthetic for metrics
    X_synth_std = scaler.transform(synth_df[feat_cols].values.astype(np.float32)).astype(np.float32)

    # Save synthetic CSV
    csv_out = f"{OUTDIR}/synthetic_aids_with_survival.csv"
    synth_df.to_csv(csv_out, index=False)

    # 3) Fidelity metrics
    MDE_per_var_df, MDE_mean_pct = marginal_KS_percent(X_real_std, X_synth_std, feat_names)
    P_corr_df, P_corr_mean_pct   = pairwise_corr_error_percent(X_real_std, X_synth_std, feat_names)
    MDE_per_var_df.to_csv(f"{OUTDIR}/marginal_errors_per_variable_aids.csv", index=False)
    P_corr_df.to_csv(f"{OUTDIR}/pairwise_corr_errors_aids.csv", index=False)
    save_table2_aids_only(MDE_mean_pct, P_corr_mean_pct)

    # 4) UMAP & Heatmaps
    umap_path = save_umap_paperstyle(X_real_std, X_synth_std, tag=tag, panel_label="(b) AIDS", xlim=(-10,15), ylim=(-5,15))
    corr_path = save_corr_heatmaps_with_labels(X_real_std, X_synth_std, feat_names, tag=tag)

    # 5) Survival KM & CoxPH confusion
    km_path = km_plot_by_treatment(full_df, synth_df, time_col=time_col, event_col=event_col, treat_col=treat_col, tag=tag)
    cm, sig_real, sig_syn = cox_significance_confusion(full_df, synth_df, time_col=time_col, event_col=event_col)
    save_table_as_image(cm, "TABLE V: CoxPH significant covariates — Confusion Matrix (AIDS)",
                        f"{OUTDIR}/table5_aids_cox_confusion.png")

    # 6) Detection scores (AIDS-only tables)
    det_log_acc, det_log_auc = detection_score_logistic(X_real_std, X_synth_std)
    save_detection_aids_only(det_log_acc, det_log_auc, title="TABLE VI: Detection Score (↑) — AIDS (Logistic)")
    det_best_acc, det_best_auc, det_best_name = detection_score_bestof(X_real_std, X_synth_std, seed=SEED)
    save_detection_aids_only(det_best_acc, det_best_auc, title=f"TABLE VI: Detection Score (↑) — AIDS (Best-of {det_best_name})")

    # Per-variable marginal error table image
    save_table_as_image(MDE_per_var_df.round({"KS%":2}),
                        "Marginal Errors per Variable — AIDS",
                        f"{OUTDIR}/marginal_errors_per_variable_aids.png")

    # Console summary
    print("\n=== AIDS (ACTG175) — FINAL METRICS ===")
    print(pd.Series({
        "Marginal KS mean %": MDE_mean_pct,
        "Pairwise |Δρ| mean %": P_corr_mean_pct,
        "Detection Acc (%) — Logistic": det_log_acc,
        "Detection AUC — Logistic": det_log_auc,
        "Detection Acc (%) — Best-of": det_best_acc,
        "Detection AUC — Best-of": det_best_auc,
        "Best-of attacker": det_best_name,
    }))
    print("Saved:")
    print(" • Synthetic CSV:", csv_out)
    print(" • UMAP:", umap_path)
    print(" • Heatmaps:", corr_path)
    print(" • Table II:", f"{OUTDIR}/table2_aids.png")
    print(" • Marginal per-variable (CSV):", f"{OUTDIR}/marginal_errors_per_variable_aids.csv")
    print(" • Pairwise errors (CSV):", f"{OUTDIR}/pairwise_corr_errors_aids.csv")
    print(" • Survival KM:", f"{OUTDIR}/survival_km_{tag.lower()}.png")
    print(" • Cox confusion:", f"{OUTDIR}/table5_aids_cox_confusion.png")
    print(" • Detection:", f"{OUTDIR}/table6_aids_detection.png")

def run_pipeline_from_ucirepo(uci_id: int = 890):
    full_df, X, colnames, tag = load_actg175_from_ucirepo(uci_id=uci_id)
    return run_pipeline_from_df(
        full_df,
        time_col=colnames["time"],
        event_col=colnames["event"],
        treat_col=colnames["treat"],
        tag=tag
    )

# ---------------------------
# CLI
# ---------------------------
if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument("--uci_id", type=int, default=890, help="UCI dataset id (default: 890 for ACTG175).")
    args, _unknown = parser.parse_known_args()
    run_pipeline_from_ucirepo(args.uci_id)


/share/apps/rc/software/Anaconda3/2023.07-2/lib/python3.11/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/share/apps/rc/software/Anaconda3/2023.07-2/lib/python3.11/site-packages/numpy/lib/function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]
/share/apps/rc/software/Anaconda3/2023.07-2/lib/python3.11/site-packages/numpy/lib/function_base.py:2854: RuntimeWarning: invalid value encountered in divide
  c /= stddev[:, None]
/share/apps/rc/software/Anaconda3/2023.07-2/lib/python3.11/site-packages/numpy/lib/function_base.py:2855: RuntimeWarning: invalid value encountered in divide
  c /= stddev[None, :]



=== AIDS (ACTG175) — FINAL METRICS ===
Marginal KS mean %               1.013643
Pairwise |Δρ| mean %              7.60523
Detection Acc (%) — Logistic    47.507788
Detection AUC — Logistic         0.465453
Detection Acc (%) — Best-of     89.106684
Detection AUC — Best-of          0.940437
Best-of attacker                   RF-400
dtype: object
Saved:
 • Synthetic CSV: /data/user/home/rkhan5/AIDS/outputs_aids/synthetic_aids_with_survival.csv
 • UMAP: /data/user/home/rkhan5/AIDS/outputs_aids/umap_aids_paperstyle.png
 • Heatmaps: /data/user/home/rkhan5/AIDS/outputs_aids/corr_heatmaps_aids_triptych.png
 • Table II: /data/user/home/rkhan5/AIDS/outputs_aids/table2_aids.png
 • Marginal per-variable (CSV): /data/user/home/rkhan5/AIDS/outputs_aids/marginal_errors_per_variable_aids.csv
 • Pairwise errors (CSV): /data/user/home/rkhan5/AIDS/outputs_aids/pairwise_corr_errors_aids.csv
 • Survival KM: /data/user/home/rkhan5/AIDS/outputs_aids/survival_km_aids.png
 • Cox confusion: /data/user/home/rk